# SVM classification SZ vs. HC 

Classify schizophrenia group from controls using cortical thickness deviation scores (z-scores) and then the true cortical thickness data to see which type of data better separates the groups.

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

In [34]:
X1 = np.load('sz_z.npy')
X2 = np.load('sz_ct.npy')
X3 =np.load('sz_ct_age_sex.npy')
X4 = np.load('sz_ct_age_sex_site.npy')
y = np.load('sz_labels.npy')

In [ ]:
n_samples, n_features = X1.shape
random_state = np.random.RandomState(0)

## Deviation scores as features

In [ ]:
cv = StratifiedKFold(n_splits=5)
n_permutations = 1000

classifier_z = svm.SVC(kernel="linear", probability=True, random_state=random_state)
classifier_z2 = svm.SVC(kernel="linear", probability=True, random_state=random_state)

tprs_z = []
aucs_z = []
tprs_z2 = []
aucs_z2 = []
aucs_z_perms = []
mean_auc_z_perms = []
aucs_z_perms_shuff = []
mean_auc_z_perms_shuff = []
mean_fpr_z = np.linspace(0, 1, 100)
mean_fpr_z2 = np.linspace(0, 1, 100)

classifier_ct = svm.SVC(kernel="linear", probability=True, random_state=random_state)
classifier_ct2 = svm.SVC(kernel="linear", probability=True, random_state=random_state)

tprs_ct = []
aucs_ct = []
tprs_ct2 = []
aucs_ct2 = []
aucs_ct_perms = []
mean_auc_ct_perms = []
aucs_ct_perms_shuff = []
mean_auc_ct_perms_shuff = []
mean_fpr_ct = np.linspace(0, 1, 100)
mean_fpr_ct2 = np.linspace(0, 1, 100)

diff_mean_auc_z_ct_perms = []
diff_mean_auc_z_ct_perms_shuff = []

fig, ax = plt.subplots(figsize=(15,15))

for perm in range(0, n_permutations):
    random_state_perm = np.random.RandomState(1)
    y_perms = shuffle(y, random_state=random_state_perm)
    
    for i, (train, test) in enumerate(cv.split(X1, y)):
        classifier_z.fit(X1[train], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_z,
            X1[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_z = np.interp(mean_fpr_z, viz.fpr, viz.tpr)
        interp_tpr_z[0] = 0.0
        tprs_z.append(interp_tpr_z)
        aucs_z.append(viz.roc_auc)

    mean_tpr_z = np.mean(tprs_z, axis=0)
    mean_tpr_z[-1] = 1.0
    mean_auc_z = auc(mean_fpr_z, mean_tpr_z)
    std_auc_z = np.std(aucs_z)
    aucs_z_perms.append(aucs_z)
    mean_auc_z_perms.append(mean_auc_z)
    
    for i, (train, test) in enumerate(cv.split(X1, y_perms)):
        classifier_z2.fit(X1[train], y_perms[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_z2,
            X1[test],
            y_perms[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_z2 = np.interp(mean_fpr_z2, viz.fpr, viz.tpr)
        interp_tpr_z2[0] = 0.0
        tprs_z2.append(interp_tpr_z2)
        aucs_z2.append(viz.roc_auc)

    mean_tpr_z2 = np.mean(tprs_z2, axis=0)
    mean_tpr_z2[-1] = 1.0
    mean_auc_z2 = auc(mean_fpr_z2, mean_tpr_z2)
    std_auc_z2 = np.std(aucs_z2)
    aucs_z_perms_shuff.append(aucs_z2)
    mean_auc_z_perms_shuff.append(mean_auc_z2)

    for i, (train, test) in enumerate(cv.split(X4, y)):
        classifier_ct.fit(X4[train], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_ct,
            X4[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_ct = np.interp(mean_fpr_ct, viz.fpr, viz.tpr)
        interp_tpr_ct[0] = 0.0
        tprs_ct.append(interp_tpr_ct)
        aucs_ct.append(viz.roc_auc)

    mean_tpr_ct = np.mean(tprs_ct, axis=0)
    mean_tpr_ct[-1] = 1.0
    mean_auc_ct = auc(mean_fpr_ct, mean_tpr_ct)
    std_auc_ct = np.std(aucs_ct)
    aucs_ct_perms.append(aucs_ct)
    mean_auc_ct_perms.append(mean_auc_ct)
    
    for i, (train, test) in enumerate(cv.split(X4, y_perms)):
        classifier_ct2.fit(X4[train], y_perms[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_ct2,
            X4[test],
            y_perms[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_ct2 = np.interp(mean_fpr_ct2, viz.fpr, viz.tpr)
        interp_tpr_ct2[0] = 0.0
        tprs_ct2.append(interp_tpr_ct2)
        aucs_ct2.append(viz.roc_auc)

    mean_tpr_ct2 = np.mean(tprs_ct2, axis=0)
    mean_tpr_ct2[-1] = 1.0
    mean_auc_ct2 = auc(mean_fpr_ct2, mean_tpr_ct2)
    std_auc_ct2 = np.std(aucs_ct2)
    aucs_ct_perms_shuff.append(aucs_ct2)
    mean_auc_ct_perms_shuff.append(mean_auc_ct2)


    diff_mean_auc_z_ct = mean_auc_z - mean_auc_ct
    diff_mean_auc_z_ct_perms.append(diff_mean_auc_z_ct)
    
    diff_mean_auc_z_ct_shuff = mean_auc_z2 - mean_auc_ct2
    diff_mean_auc_z_ct_perms_shuff.append(diff_mean_auc_z_ct_shuff)
    plt.close()
    
diff_shuff = np.array(diff_mean_auc_z_ct_perms_shuff)
diff_true = np.array(diff_mean_auc_z_ct_perms)

pvalue = (np.sum(diff_shuff >= diff_true) + 1.0) / (n_permutations + 1)
print("Permutation p-value is: ", pvalue)

In [ ]:
cv = StratifiedKFold(n_splits=5)
n_permutations = 1000

classifier_z = svm.SVC(kernel="linear", probability=True, random_state=random_state)
classifier_z2 = svm.SVC(kernel="linear", probability=True, random_state=random_state)

tprs_z = []
aucs_z = []
tprs_z2 = []
aucs_z2 = []
aucs_z_perms = []
mean_auc_z_perms = []
aucs_z_perms_shuff = []
mean_auc_z_perms_shuff = []
mean_fpr_z = np.linspace(0, 1, 100)
mean_fpr_z2 = np.linspace(0, 1, 100)

classifier_ct = svm.SVC(kernel="linear", probability=True, random_state=random_state)
classifier_ct2 = svm.SVC(kernel="linear", probability=True, random_state=random_state)

tprs_ct = []
aucs_ct = []
tprs_ct2 = []
aucs_ct2 = []
aucs_ct_perms = []
mean_auc_ct_perms = []
aucs_ct_perms_shuff = []
mean_auc_ct_perms_shuff = []
mean_fpr_ct = np.linspace(0, 1, 100)
mean_fpr_ct2 = np.linspace(0, 1, 100)

diff_mean_auc_z_ct_perms = []
diff_mean_auc_z_ct_perms_shuff = []

fig, ax = plt.subplots(figsize=(15,15))

for perm in range(0, n_permutations):
    random_state_perm = np.random.RandomState(1)
    y_perms = shuffle(y, random_state=random_state_perm)
    
    for i, (train, test) in enumerate(cv.split(X1, y)):
        classifier_z.fit(X1[train], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_z,
            X1[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_z = np.interp(mean_fpr_z, viz.fpr, viz.tpr)
        interp_tpr_z[0] = 0.0
        tprs_z.append(interp_tpr_z)
        aucs_z.append(viz.roc_auc)

    mean_tpr_z = np.mean(tprs_z, axis=0)
    mean_tpr_z[-1] = 1.0
    mean_auc_z = auc(mean_fpr_z, mean_tpr_z)
    std_auc_z = np.std(aucs_z)
    aucs_z_perms.append(aucs_z)
    mean_auc_z_perms.append(mean_auc_z)
    
    for i, (train, test) in enumerate(cv.split(X1, y_perms)):
        classifier_z2.fit(X1[train], y_perms[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_z2,
            X1[test],
            y_perms[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_z2 = np.interp(mean_fpr_z2, viz.fpr, viz.tpr)
        interp_tpr_z2[0] = 0.0
        tprs_z2.append(interp_tpr_z2)
        aucs_z2.append(viz.roc_auc)

    mean_tpr_z2 = np.mean(tprs_z2, axis=0)
    mean_tpr_z2[-1] = 1.0
    mean_auc_z2 = auc(mean_fpr_z2, mean_tpr_z2)
    std_auc_z2 = np.std(aucs_z2)
    aucs_z_perms_shuff.append(aucs_z2)
    mean_auc_z_perms_shuff.append(mean_auc_z2)

    for i, (train, test) in enumerate(cv.split(X4, y)):
        classifier_ct.fit(X4[train], y[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_ct,
            X4[test],
            y[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_ct = np.interp(mean_fpr_ct, viz.fpr, viz.tpr)
        interp_tpr_ct[0] = 0.0
        tprs_ct.append(interp_tpr_ct)
        aucs_ct.append(viz.roc_auc)

    mean_tpr_ct = np.mean(tprs_ct, axis=0)
    mean_tpr_ct[-1] = 1.0
    mean_auc_ct = auc(mean_fpr_ct, mean_tpr_ct)
    std_auc_ct = np.std(aucs_ct)
    aucs_ct_perms.append(aucs_ct)
    mean_auc_ct_perms.append(mean_auc_ct)
    
    for i, (train, test) in enumerate(cv.split(X4, y_perms)):
        classifier_ct2.fit(X4[train], y_perms[train])
        viz = RocCurveDisplay.from_estimator(
            classifier_ct2,
            X4[test],
            y_perms[test],
            name="ROC fold {}".format(i),
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr_ct2 = np.interp(mean_fpr_ct2, viz.fpr, viz.tpr)
        interp_tpr_ct2[0] = 0.0
        tprs_ct2.append(interp_tpr_ct2)
        aucs_ct2.append(viz.roc_auc)

    mean_tpr_ct2 = np.mean(tprs_ct2, axis=0)
    mean_tpr_ct2[-1] = 1.0
    mean_auc_ct2 = auc(mean_fpr_ct2, mean_tpr_ct2)
    std_auc_ct2 = np.std(aucs_ct2)
    aucs_ct_perms_shuff.append(aucs_ct2)
    mean_auc_ct_perms_shuff.append(mean_auc_ct2)


    diff_mean_auc_z_ct = mean_auc_z - mean_auc_ct
    diff_mean_auc_z_ct_perms.append(diff_mean_auc_z_ct)
    
    diff_mean_auc_z_ct_shuff = mean_auc_z2 - mean_auc_ct2
    diff_mean_auc_z_ct_perms_shuff.append(diff_mean_auc_z_ct_shuff)
    plt.close()
    
diff_shuff = np.array(diff_mean_auc_z_ct_perms_shuff)
diff_true = np.array(diff_mean_auc_z_ct_perms)

pvalue = (np.sum(diff_shuff >= diff_true) + 1.0) / (n_permutations + 1)
print("Permutation p-value is: ", pvalue)

In [ ]:
diff_shuff = np.array(diff_mean_auc_z_ct_perms_shuff)
diff_true = np.array(diff_mean_auc_z_ct_perms)

In [ ]:
pvalue = (np.sum(diff_shuff >= diff_true) + 1.0) / (n_permutations + 1)